In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import numpy as np
import pandas as pd
import os
import gzip
import tensorflow as tf
from tensorflow import keras
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt

常量

In [38]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.,]])
# index
print(t)
print(t[:, 1:])
print(t[:, 1])  # 只取出第二列

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [6]:
# op
print(t + 10)
print(tf.square(t))
print(t @ tf.transpose(t)) # tf.transpose()转置  @ = np.dot

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [9]:
# numpy conversion
print(t.numpy())
print(np.square(t))
np_t = np.array([[1., 2., 3.], [4., 5., 6.]])
print(tf.constant(np_t))

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [39]:
# 0维tensor  ->scalars
t = tf.constant(2.718)
print(t.shape)
print(t.numpy())
print(t.shape)

()
2.718
()


In [17]:
# strings
t = tf.constant("cafe")
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t, unit='UTF8_CHAR'))    # unit的输入只能是BYTE或UTF8_CHAR
print(tf.strings.unicode_decode(t, 'UTF8'))
# print(tf.strings.unicode_decode(t, 'utf-8'))   unicode_decode可以用utf-8

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [18]:
t = tf.constant(["cafe", "coffee", "咖啡"])
print(tf.strings.length(t, unit='UTF8_CHAR'))
print(tf.strings.unicode_decode(t, 'UTF8'))

tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


In [24]:
# RaggedTensor
r = tf.ragged.constant([[11, 12], [21, 22, 23], [1], [41]])
# index op
print(r)
print(r[1])
print(r[1:2]) # 左闭右开， 二维
print(r[1:3])

<tf.RaggedTensor [[11, 12], [21, 22, 23], [1], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23]]>
<tf.RaggedTensor [[21, 22, 23], [1]]>


In [ ]:
# ops on ragged tensor
r2 = tf.ragged.constant([[51, 52], [], [71]])
print(tf.concat([r, r2], axis=0))
# print(tf.concat([r, r2], axis=1)) InvalidArgumentError: Input tensors have incompatible shapes.

In [28]:
# 横向拼接需要（横向）有一样的维数
r3 = tf.ragged.constant([[13, 14], [15], [], [42, 43]])
print(tf.concat([r, r3], axis=1))

<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 15], [1], [41, 42, 43]]>


In [29]:
# ragged tensor -> tensor  
# 在正常值的 【后面】 用0补齐
print(r.to_tensor())

tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 1  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)


In [30]:
# sparse tensor  记录非零位置及对应的值
s = tf.SparseTensor(indices = [[0, 1], [1, 0], [2, 3]],
                    values = [1., 2., 3.], dense_shape = [3, 4])
print(s)  # s该不是tensor，只是记录了生成sparse法则
print(tf.sparse.to_dense(s))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [31]:
# ops on sparse tensors
s2 = s * 2.0
print(s2)               # 非零位置不变， 对应的值变了

try:
    s3 = s + 1          # sparse不支持加法      
except TypeError as ex:
    print(ex)
    
s4 = tf.constant([[10., 20.],
                  [30., 40.],
                  [40., 60.],
                  [70., 80.]])
print(tf.sparse.sparse_dense_matmul(s, s4))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'int'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


In [34]:
# to_dense需要定义SparseTensor时indices是排好顺序的
s5 = tf.SparseTensor(indices = [[0, 2], [0, 1], [2, 3]],
                    values = [1., 2., 3.], dense_shape = [3, 4])
print(s5)
# print(tf.sparse.to_dense(s5))

# 解决reorder
s6 = tf.sparse.reorder(s5)
print(tf.sparse.to_dense(s6))

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 2. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


变量

In [36]:
# Variables
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print(v)
print(v.value())  # 具体的tensor 就相当于constant
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


In [37]:
# 操作跟常量差不多  多一些重新赋值
# assign value  不能用 【=】
v.assign(2 * v)
print(v.numpy())
v[0, 1].assign(42)
print(v.numpy())
v[1].assign([7., 8., 9.])
print(v.numpy())

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

[[ 2.  4.  6.]
 [ 8. 10. 12.]]


<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

[[ 2. 42.  6.]
 [ 8. 10. 12.]]


<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 7.,  8.,  9.]], dtype=float32)>

[[ 2. 42.  6.]
 [ 7.  8.  9.]]
